In [3]:
# !pip install annoy

In [4]:
# -- coding: utf-8 -

In [9]:
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [101]:
def preprocess_txt(line):   
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    
    return spls

In [102]:
# assert False

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w", encoding='utf8') as fout:
    with open("Otvety.txt", "r", encoding='utf8') as fin:
        for line in tqdm_notebook(fin):
            # первая строка пустая. из-за этого получается что первый вопрос записывается в файл без ответа
            if len(line) < 3:
                continue
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                spls = preprocess_txt(line)
                # отбросим слишком короткие ответы
                if len(spls) > 3:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                continue
            if not written:
                question = line.strip()
                continue

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [103]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r", encoding='utf8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [104]:
sentences = [i for i in sentences if len(i) > 2]

In [168]:
# sg = 1 skip-gram, hs = 0 + negative = 10 negative sampling, window=2 окно 2 слова, size=200 размер вектора 200
# sample = 1е-5 субсэмплирование 1е-5
modelW2V = Word2Vec(sentences=sentences,size=200, window=2, min_count=1, sg = 1, hs = 0, negative = 10, sample = 1e-5, workers= 32, seed = 25)

In [169]:
modelFT = FastText(sentences=sentences,size=200, window=2, min_count=1, sg = 1, hs = 0, negative = 10, sample = 1e-5, workers= 32, seed = 25)

In [ ]:
# vect_idf = TfidfVectorizer()
# vect_idf.fit_transform(df_train['Content'])
# tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

w2v_index = annoy.AnnoyIndex(200 ,'angular')
ft_index = annoy.AnnoyIndex(200 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding='utf8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        # vector_w2v = np.zeros(50)
        # vector_ft = np.zeros(50)
        vector_w2v = np.zeros(200)
        vector_ft = np.zeros(200)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 200000:
            break

w2v_index.build(10)
ft_index.build(10)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [159]:
modelW2V.save('modelw2v.bin')

In [160]:
modelFT.save('modelft.bin')

In [165]:
w2v_index.save('w2v_index.bin')

True

In [166]:
ft_index.save('ft_index.bin')

True

In [167]:
pickle.dump(index_map, open('index_map.pkl', 'wb'))

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "когда сменится президент?"
get_response(TEXT, w2v_index, modelW2V, index_map)

In [ ]:
TEXT = "когда сменится президент?"
get_response(TEXT, ft_index, modelFT, index_map)

In [149]:
TEXT = "какая сегодня погода?"
get_response(TEXT, w2v_index, modelW2V, index_map)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


['у нас жара ужасная,поэтому дома сидим.. \n',
 'Ну если не считать ветра, холодных градинок прям в лицо и то,что 3 раза сегодня упала, то да, ничего так погода.:). \n',
 'Уважаемая Людо Перелыгина, здравствуйте! Ваш вопрос относится к разделам Экологической Метеорологии. На поверхности кожи микрокапилляры (поры) постоянно в результате нормального функционирования, выделяют пот - отходы жизнедеятельности организма, которые выглядят в виде капелек. Пот, кроме того, имеет важную функцию - охлаждение организма при перегреве за счёт испарения влаги с поверхности кожи; вот почему когда человеку жарко (или душно) потоотделение резко усиливается. Так что личное ощущение "прохлады" от работы вентилятора связано с тем, что скорость испарения влаги (с любой поверхности и с воды) резко возрастает при увеличении скорости движении воздуха над испаряющей поверхностью и при этом теплоотдача на испарение резко увеличивается и поверхность кожи это "чувствует" в виде прохлады. При этом, что касается пот

In [150]:
TEXT = "какая сегодня погода?"
get_response(TEXT, ft_index, modelFT, index_map)[0]

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


'Это очень хорошо. Если умеючи. Если присутствует не только влечение. Это даже пользительно для организма. Это - продолжение рода. Да и вообще, мы же люди! Это у остальных живых существ - размножение, а у нас ещё и удовольствие, КАЙФ!. \n'

In [117]:
TEXT = "как дела?"
get_response(TEXT, ft_index, modelFT, index_map)[0]

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


'думу думаю.. поэтому они в отсрочке)). \n'

In [118]:
TEXT = "как дела?"
get_response(TEXT, w2v_index, modelW2V, index_map)

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


['думу думаю.. поэтому они в отсрочке)). \n',
 'Привет! Настроение если честно, так себе.. Сейчас будем поднимать :-)<br>Наконец зашел на „Ответы”... Здесь можно забыть о будничных проблемах, а заодно помочь другим... Что может быть приятнее?<br>Хорошего настроения и удачного Вам дня!<br>. \n',
 'является ли помощник депутата госслужащим ? . \n']